<a href="https://colab.research.google.com/github/foadrezei/Enigma-Machine-Simulator-in-Python/blob/main/enigma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class EnigmaMachine:
    def __init__(self, rotor_settings, plugboard_settings):
        # Alphabet for mapping
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

        # Rotor wirings (simplified, each rotor is a permutation of the alphabet)
        self.rotors = [
            "EKMFLGDQVZNTOWYHXUSPAIBRCJ",  # Rotor I
            "AJDKSIRUXBLHWTMCQGZNPYFVOE",  # Rotor II
            "BDFHJLCPRTXVZNYEIWGAKMUSQO"   # Rotor III
        ]

        # Reflector (UKW-B, pairs letters)
        self.reflector = "YRUHQSLDPXNGOKMIEBFZCWVJAT"

        # Rotor positions (0-25, set by user)
        self.rotor_positions = rotor_settings  # e.g., [0, 0, 0]

        # Plugboard (list of tuples for letter swaps, e.g., [("A", "B"), ("C", "D")])
        self.plugboard = plugboard_settings

    def rotate_rotors(self):
        # Rotate the rightmost rotor
        self.rotor_positions[2] = (self.rotor_positions[2] + 1) % 26
        # Check for carry-over to the middle rotor
        if self.rotor_positions[2] == 0:
            self.rotor_positions[1] = (self.rotor_positions[1] + 1) % 26
            # Check for carry-over to the left rotor
            if self.rotor_positions[1] == 0:
                self.rotor_positions[0] = (self.rotor_positions[0] + 1) % 26

    def plugboard_swap(self, char):
        # Swap letters according to plugboard settings
        for pair in self.plugboard:
            if char == pair[0]:
                return pair[1]
            if char == pair[1]:
                return pair[0]
        return char

    def rotor_encrypt(self, char_idx, rotor_idx, forward=True):
        # Get the rotor wiring
        rotor = self.rotors[rotor_idx]
        # Adjust for rotor position
        pos = (char_idx + self.rotor_positions[rotor_idx]) % 26 if forward else (char_idx - self.rotor_positions[rotor_idx]) % 26
        # Map through the rotor
        if forward:
            mapped_char = rotor[pos]
            return self.alphabet.index(mapped_char)
        else:
            mapped_char = self.alphabet[pos]
            return rotor.index(mapped_char)

    def reflector_encrypt(self, char_idx):
        # Reflector maps the input to its paired letter
        mapped_char = self.reflector[char_idx]
        return self.alphabet.index(mapped_char)

    def encrypt_char(self, char):
        if char not in self.alphabet:
            return char  # Ignore non-alphabetic characters

        # Step 1: Rotate rotors
        self.rotate_rotors()

        # Step 2: Plugboard (initial swap)
        char = self.plugboard_swap(char)
        char_idx = self.alphabet.index(char)

        # Step 3: Forward pass through rotors (right to left)
        for i in range(2, -1, -1):  # Rotor 3 -> 2 -> 1
            char_idx = self.rotor_encrypt(char_idx, i, forward=True)

        # Step 4: Reflector
        char_idx = self.reflector_encrypt(char_idx)

        # Step 5: Backward pass through rotors (left to right)
        for i in range(3):  # Rotor 1 -> 2 -> 3
            char_idx = self.rotor_encrypt(char_idx, i, forward=False)

        # Step 6: Plugboard (final swap)
        char = self.alphabet[char_idx]
        char = self.plugboard_swap(char)

        return char

    def encrypt(self, message):
        # Convert message to uppercase and encrypt each character
        message = message.upper()
        encrypted = ""
        for char in message:
            encrypted += self.encrypt_char(char)
        return encrypted

if __name__ == "__main__":
    # Example usage
    rotor_settings = [0, 0, 0]  # Initial rotor positions
    plugboard_settings = [("A", "B"), ("C", "D")]  # Plugboard swaps
    enigma = EnigmaMachine(rotor_settings, plugboard_settings)

    message = "HELLO"
    encrypted = enigma.encrypt(message)
    print(f"Original: {message}")
    print(f"Encrypted: {encrypted}")

    # To decrypt, reset the rotor positions and use the same settings
    enigma = EnigmaMachine(rotor_settings, plugboard_settings)
    decrypted = enigma.encrypt(encrypted)
    print(f"Decrypted: {decrypted}")